# <span style="color:red">**exercise 5**</span>
---

### submited by:
- Name: Shahar Asher
- Id: 209305408
- Email adress: shaharas@edu.hac.ac.il
- Date: 16/05/2024

### Operation system: Windows 11
### Python version: 3.11.5 (Using anaconda)
### IDE: Visual Studio Code
### libraries: typing, re, time, inspect, functools, contextlib
---

In [19]:
# imports

from typing import Final
import re
import time
import inspect
from functools import wraps
from contextlib import ContextDecorator

# Q.1

Write a decorator that measures a function's wall and CPU consumption (You may use `perf_counter` and `process_counter`).
- The decorator should output to a standard output or to a log file.
- The output must include the following data:
- The function name
- File-name and line-number where the function is being called (Hint: use the `inspect` module).
- A counter counting the function calls
- The CPU time consumed at each function call (time resolution 1μs (1E-6) seconds)
- The wall time (time resolution 10μs (1E-5) seconds)
For example, if the function `func1` is being called 3 times and `func2` once (both functions decorated with your decorator), one may get:
```
Function name  filename    line  count  cpu        wall
func1          mymod.py    4     1      0.123456   0.123721
func1          mymod.py    8     2      0.027010   0.030002
func2          mymod.py    6     1      1.301104   0.056213
func1          mymod.py    4     3      0.117010   0.110002
```
- Make the log output switchable by a global variable `PROFILE`.

---
Global variables to control profiling and header printing.

- `PROFILE`: A boolean to enable or disable profiling.
- `HEADER_PRINTER`: A boolean to control the printing of the header.

In [20]:
# global variables
PROFILE:bool = True
HEADER_PRINTER:bool = False

In [21]:
def profile_decorator(func:callable) -> callable:
    """
    A decorator that profiles the execution time of the decorated function.
    
    Parameters:
    func (callable): The function to be decorated.

    Returns:
    callable: The wrapped function with profiling.
    """
    # Initialize a counter for function calls
    func._call_count = 0

    @wraps(func)
    def wrapper(*args, **kwargs) -> callable:
        """
        Wrapper function that performs profiling.

        Parameters:
        *args: Variable length argument list.
        **kwargs: Arbitrary keyword arguments.

        Returns:
        callable: The result of the wrapped function.
        """
        global HEADER_PRINTER
        
        if not PROFILE:
            return func(*args, **kwargs)

        # Increment the function call counter
        func._call_count += 1

        # Get the frame information of the caller
        frame:Final[object] = inspect.currentframe().f_back
        filename:str = frame.f_code.co_filename
        filename = re.search(r"(\w+.py)", filename)
        filename = filename.group()
        lineno:Final[int] = frame.f_lineno

        # Start measuring CPU and wall time
        start_cpu:Final[float] = time.process_time_ns()
        start_wall:Final[float] = time.perf_counter_ns()

        # Execute the function
        result = func(*args, **kwargs)

        # Stop measuring CPU and wall time
        end_cpu:Final[float] = time.process_time_ns()
        end_wall:Final[float] = time.perf_counter_ns()

        # Calculate the elapsed times
        cpu_time:Final[float] = (end_cpu - start_cpu) / 1e9  # Convert nanoseconds to seconds
        wall_time:Final[float] = (end_wall - start_wall) / 1e9  # Convert nanoseconds to seconds

        # Log the details in table format
        if not HEADER_PRINTER:
            print(f"{'Function name':<15} {'filename':<15} {'line':<5} {'count':<5} {'cpu':<10} {'wall':<10}")
            HEADER_PRINTER = True

        print(f"{func.__name__:<15} {filename:<15} {lineno:<5} {func._call_count:<5} {cpu_time:<10.6f} {wall_time:<10.5f}")

        return result

    return wrapper

Define two sample functions `func1` and `func2` to demonstrate the profiling decorator.

- `func1`: A computation-intensive function to demonstrate profiling.
- `func2`: Another computation-intensive function to demonstrate profiling.

In [22]:
@profile_decorator
def func1() -> None:
    """
    A sample function to demonstrate profiling. 
    It performs a computation-intensive task.
    """
    total:int = 0
    for _ in range(1000000):
        total += sum(i * i for i in range(100))


@profile_decorator
def func2() -> None:
    """
    Another sample function to demonstrate profiling. 
    It performs a computation-intensive task.
    """
    total:int = 0
    for _ in range(1000000):
        total += sum(i * i for i in range(100))

Call the functions to observe the profiling output.

- Call `func1` three times.
- Call `func2` twice.

In [23]:
# call to func1
func1()
func1()
func1()

# call to func2
func2()
func2()

Function name   filename        line  count cpu        wall      
func1           2948867115.py   2     1     5.125000   6.89332   
func1           2948867115.py   3     2     4.437500   9.47010   
func1           2948867115.py   4     3     4.687500   9.41312   
func2           2948867115.py   7     1     4.531250   8.70295   
func2           2948867115.py   8     2     5.203125   6.23485   


---
# Q.2

Make a context manager that does the same thing.

The idea here is to profile a block of code inside a function or a script's main body.

---
Define a `ProfileBlock` class that acts as both a context manager and a decorator for profiling blocks of code or functions.

- `ProfileBlock`: A context manager and decorator class for profiling blocks of code or functions.
- `func_name`: The name of the function or block being profiled.

In [24]:
# global variables
PROFILE:bool = True
HEADER_PRINTER:bool = False

In [25]:
class ProfileBlock(ContextDecorator):
    """
    A context manager and decorator class for profiling blocks of code or functions.
    
    Attributes:
    func_name (str): The name of the function or block being profiled.
    """

    def __init__(self, func_name:str="ProfileBlock") -> None:
        """
        Initializes the ProfileBlock with a function or block name.

        Parameters:
        func_name (str): The name of the function or block being profiled.
        """
        self.func_name:str = func_name


    def __enter__(self) -> object:
        """
        Enters the runtime context for profiling.
        
        Returns:
        self: The context manager instance.
        """
        global HEADER_PRINTER

        if not PROFILE:
            return self

        # Get the frame information of the caller
        self.frame:object = inspect.currentframe().f_back
        self.filename:str = self.frame.f_code.co_filename
        self.filename = re.search(r"(\w+.py)", self.filename)
        self.filename = self.filename.group()
        self.lineno:int = self.frame.f_lineno

        # Start measuring CPU and wall time
        self.start_cpu:float = time.process_time()
        self.start_wall:float = time.perf_counter()

        # Initialize call count
        if not hasattr(self, 'call_count'):
            self.call_count:int = 0

        return self
    
    
    def __exit__(self, exc_type:object, exc_value:object, traceback:object) -> None:
        """
        Exits the runtime context and logs the profiling information.

        Parameters:
        exc_type (type): The exception type, if an exception occurred.
        exc_value (Exception): The exception instance, if an exception occurred.
        traceback (traceback): The traceback object, if an exception occurred.
        """
        if not PROFILE:
            return

        # Stop measuring CPU and wall time
        end_cpu:float = time.process_time()
        end_wall:float = time.perf_counter()

        # Calculate the elapsed times
        cpu_time:float = end_cpu - self.start_cpu
        wall_time:float = end_wall - self.start_wall

        # Increment the call count
        self.call_count += 1

        # Print the header if not already printed
        global HEADER_PRINTER
        if not HEADER_PRINTER:
            print(f"{'Function name':<15} {'filename':<15} {'line':<5} {'count':<5} {'cpu':<10} {'wall':<10}")
            HEADER_PRINTER = True

        # Log the details in table format
        print(f"{self.func_name:<15} {self.filename:<15} {self.lineno:<5} {self.call_count:<5} {cpu_time:<10.6f} {wall_time:<10.5f}")

Define two sample functions `func1` and `func2` to demonstrate the profiling context manager.

- `func1`: A computation-intensive function to demonstrate profiling using `ProfileBlock`.
- `func2`: Another computation-intensive function to demonstrate profiling using `ProfileBlock`.

In [26]:
def func1() -> None:
    """
    A sample function to demonstrate profiling using ProfileBlock context manager. 
    It performs a computation-intensive task.
    """
    with ProfileBlock("func1"):
        total:int = 0
        for _ in range(1000000):
            total += sum(i * i for i in range(100))


def func2() -> None:
    """
    Another sample function to demonstrate profiling using ProfileBlock context manager. 
    It performs a computation-intensive task.
    """
    with ProfileBlock("func2"):
        total:int = 0
        for _ in range(1000000):
            total += sum(i * i for i in range(100))

Use the `ProfileBlock` context manager to profile a main block of code.

- Call `func1` three times.
- Call `func2` twice.

In [27]:
with ProfileBlock("main block"):
    """
    Main block of code demonstrating profiling using ProfileBlock context manager.
    """
    # call to func1
    func1()
    func1()
    func1()

    # call to func2
    func2()
    func2()

Function name   filename        line  count cpu        wall      
func1           2318137732.py   6     1     5.265625   6.82142   
func1           2318137732.py   6     1     4.781250   7.36649   
func1           2318137732.py   6     1     5.656250   6.56779   
func2           2318137732.py   17    1     5.296875   5.88050   
func2           2318137732.py   17    1     5.312500   5.83010   
main block      708578946.py    1     1     26.343750  32.46745  
